# COGS 108 - Data Checkpoint

# Names

- Cassandra Henrickson
- Daniel Han
- Ezra Lu
- Jasmine Qiang
- Wing Leung (Calvin)

<a id='research_question'></a>
# Research Question

How is the severity of the traffic jam due to a car crash impacted as a result of where the accident occurs and different weather conditions?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: US Accidents 
- Link to the dataset: https://www.kaggle.com/sobhanmoosavi/us-accidents?select=US_Accidents_Dec20.csv
- Number of observations: 4232541 

1-2 sentences describing each dataset. 

This dataset is a collection of car accident data across 49 US states from February 2016 to December 2020, taken from traffic APIs. It describes different variables present at each accident, including weather condition, severity of traffic jam caused by the accident, time of the accident (via start and end of the traffic jam), and location. 


If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [18]:
## YOUR CODE HERE
# Import seaborn and apply its plotting styles
import seaborn as sns
sns.set(style="white", font_scale=2)

# import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
# set plotting size parameter
plt.rcParams['figure.figsize'] = (17, 7)

# make sure pandas & numpy are imported
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'


In [19]:
df = pd.read_csv('US_Accidents_Dec20.csv')

In [20]:
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [21]:
print('Total observations: ', len(df))

Total observations:  4229394


# Data Cleaning

Describe your data cleaning steps here.

In [22]:
#49 different variables
df.columns

Index(['ID', 'Source', 'TMC', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)',
       'Description', 'Number', 'Street', 'Side', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
       'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [23]:
# San Diego has 29416 accidents
a = df[df['City'] == 'San Diego']
accident_sandiego = a['City'].count()
accident_sandiego

29401

In [24]:
#slicing df to only include columns we need - Severity, State, Weather Condition, No Exit, Junction, Stop, and Traffic Signal
df = df.loc[:, ['Severity', 'City', 'Weather_Condition', 'No_Exit', 'Junction', 'Stop', 'Traffic_Signal']]

#slicing to only get car accidents that happened in San Diego
df = df.loc[df['City'] == 'San Diego']

In [25]:
#Combine 'Light Rain' to 'Rain' and 'Cloudy' to 'Overcast'
df['Weather_Condition'] = df['Weather_Condition'].replace(['Light Rain'], 'Rain')
df['Weather_Condition'] = df['Weather_Condition'].replace(['Cloudy'], 'Overcast')

In [26]:
#slicing df to only include car accident observations with certain weather conditions
df = df.loc[(df['Weather_Condition'] =='Rain') \
            | (df['Weather_Condition'] == 'Fog') \
            | (df['Weather_Condition'] == 'Overcast') \
            | (df['Weather_Condition'] == 'Clear')].reset_index()

In [27]:
#check for missing data                                                              
print(df['Severity'].hasnans)
print(df['City'].hasnans)
print(df['Weather_Condition'].hasnans)
print(df['No_Exit'].hasnans)
print(df['Junction'].hasnans)
print(df['Stop'].hasnans)
print(df['Traffic_Signal'].hasnans)

False
False
False
False
False
False
False


In [28]:
#drop index column
df = df.drop(columns = ['index'])

#show cleaned car accident dataset
df

,Severity,City,Weather_Condition,No_Exit,Junction,Stop,Traffic_Signal
0,3,San Diego,Overcast,False,True,False,False
1,2,San Diego,Overcast,False,False,False,False
2,2,San Diego,Clear,False,False,False,False
3,3,San Diego,Clear,False,True,False,False
4,3,San Diego,Clear,False,False,False,False
...,...,...,...,...,...,...,...
11289,3,San Diego,Overcast,False,False,False,False
11290,3,San Diego,Overcast,False,False,False,False
11291,2,San Diego,Fog,False,False,False,False
11292,3,San Diego,Overcast,False,False,False,False


# Project Proposal (updated)

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  7 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/27  |  7 PM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/3  | 7 PM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/08  | 7 PM    |  finish finding data and writing code, etc | Discuss/edit, complete check-in 1: Data|
| 2/17  | 7 PM    | Import & Wrangle Data; EDA | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/24  | 7 PM   | Finalize wrangling/EDA; Begin Analysis  | Discuss/edit Analysis; Complete project check-in: EDA |
| 3/03  | 7 PM   | more analysis; Draft results/conclusion/discussion | Discuss/edit full project |
| 3/10  | 7 PM   | Complete analysis; Draft results/conclusion/discussion | Discuss/edit full project |
| 3/17  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |